<a href="https://colab.research.google.com/github/shahmedh/DEPI-group6/blob/main/Data_Collection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Task one:**

# **Data Collection**

1.   Download Sentinel-2 satellite images
2.   Ensure the data includes multispectral images that capture various spectral bands



**Installing and  Importing necessary packages**


In [ ]:
!pip install sentinelhub numpy geopandas shapely rasterio
import requests
from tqdm import tqdm
import rasterio
import numpy as np
import matplotlib.pyplot as plt
import zipfile
import os


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.8/249.8 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 58.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.0/236.0 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.3 MB/s eta 0:00:00
  Created wheel for utm: filename=utm-0.8.0-py3-none-any.whl size=8536 sha256=6751cda0f1a474c2290cd3d7645ce3a91e53949d655ee6eeeab3f08ccab9597d
  Stored in directory: /root/.cache/pip/wheels/b0/88/9b/1968d3a5933864f7d6312d8217027dd4d52b9647f678c67023
Successfully built utm


**Open a connection with the DATASPACE.COPERNICUS SERVER using Token URL, Username and Password**



In [ ]:
import requests

# Copernicus OAuth2 Token URL
TOKEN_URL = "https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token"

# Replace with your Copernicus account credentials
EMAIL = "shaimaa.ahmed.business@gmail.com"
PASSWORD = "DEPI2yth!!2025"

# Request payload
payload = {
    "client_id": "cdse-public",
    "username": EMAIL,
    "password": PASSWORD,
    "grant_type": "password"
}

# Request token
response = requests.post(TOKEN_URL, data=payload)

# Extract token if request is successful
if response.status_code == 200:
    ACCESS_TOKEN = response.json().get("access_token")
    print("Access Token retrieved successfully!")
else:
    print(f"Error: {response.status_code}, {response.text}")

Access Token retrieved successfully!


# Downloading zipped data



In [ ]:
##import requests
##from tqdm import tqdm

# Product download URL (Replace with your actual product URL)
PRODUCT_URL = "https://download.dataspace.copernicus.eu/odata/v1/Products(2fd9e578-5865-40e8-b3e9-0487d53011c3)/$value"

# Output file name
output_file = "Sentinel2_Image.zip"

# Set Authorization header
headers = {
    "Authorization": f"Bearer {ACCESS_TOKEN}"
}

# Download the file
with requests.get(PRODUCT_URL, headers=headers, stream=True) as r:
    r.raise_for_status()  # Check for request errors
    total_size = int(r.headers.get('content-length', 0))
    block_size = 1024  # 1 KB chunks
    t = tqdm(total=total_size, unit="B", unit_scale=True)

    with open(output_file, "wb") as f:
        for chunk in r.iter_content(chunk_size=block_size):
            f.write(chunk)
            t.update(len(chunk))

print(f"\nDownload completed: {output_file}")

100%|█████████▉| 1.25G/1.25G [10:51<00:00, 539kB/s]


Download completed: Sentinel2_Image.zip


# **Ensure the data includes multispectral images that capture various spectral bands**


In [ ]:
import rasterio
import numpy as np
import zipfile
import os
import gc

# Clean RAM before execution
gc.collect()

# Step 1: Extract the Sentinel-2 zip file
zip_file = "Sentinel2_Image.zip"
extract_folder = "Sentinel2_Data"

with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall(extract_folder)

# Step 2: Locate Sentinel-2 Bands
band_files = {}
band_names = {
    "B01": "Coastal Aerosol",
    "B02": "Blue",
    "B03": "Green",
    "B04": "Red",
    "B05": "Vegetation Red Edge 1",
    "B06": "Vegetation Red Edge 2",
    "B07": "Vegetation Red Edge 3",
    "B08": "NIR",
    "B8A": "Narrow NIR",
    "B09": "Water Vapour",
    "B10": "SWIR - Cirrus",
    "B11": "SWIR 1",
    "B12": "SWIR 2"
}

for root, dirs, files in os.walk(extract_folder):
    for file in files:
        for band in band_names.keys():
            if band in file and file.endswith(".jp2"):
                band_files[band] = os.path.join(root, file)

# Step 3: Read Sentinel-2 Bands
def read_band(file_path):
    with rasterio.open(file_path) as src:
        return src.read(1).astype(np.float32)

# Read all bands
bands_data = {band: read_band(path) for band, path in band_files.items()}

# Print band names and their symbols
for band, description in band_names.items():
    print(f"{band}: {description}")

# Clean up memory
del bands_data, band_files
gc.collect()

100%|██████████| 1.25G/1.25G [11:10<00:00, 539kB/s]

B01: Coastal Aerosol
B02: Blue
B03: Green
B04: Red
B05: Vegetation Red Edge 1
B06: Vegetation Red Edge 2
B07: Vegetation Red Edge 3
B08: NIR
B8A: Narrow NIR
B09: Water Vapour
B10: SWIR - Cirrus
B11: SWIR 1
B12: SWIR 2


0

# **The notebook is crashed after this step so I redo the work again without enshuring the image has all bands**